In [4]:
import cv2
import numpy

In [5]:
video = cv2.VideoCapture(r"C:\Users\hp\Videos\Screen Recordings\ball.mp4")


In [6]:
ret, prev_frame = video.read()
if not ret:
    print("Unable to read video")
    video.release()
    exit()

# Convert first frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)

# Detect good features to track
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=None, maxCorners=10, qualityLevel=0.3, minDistance=10, blockSize=7)

# Create a mask image for drawing
mask = numpy.zeros_like(prev_frame)

while True:
    ret, current_frame = video.read()
    if not ret:
        break

    # Convert to grayscale
    current_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    current_corners, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, current_gray, prev_corners, None, **lk_params)

    # Select good points
    good_new = current_corners[status == 1]
    good_old = prev_corners[status == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        current_frame = cv2.circle(current_frame, (int(a), int(b)), 5, (0, 0, 255), -1)

    # Overlay the mask on the frame
    output = cv2.add(current_frame, mask)

    # Show the output
    cv2.imshow("Optical Flow", output)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    # Update previous frame and points
    prev_gray = current_gray.copy()
    prev_corners = good_new.reshape(-1, 1, 2)

video.release()
cv2.destroyAllWindows()
